In [14]:
import pandas as pd
import torch
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pprint import pprint as print
import seaborn as sns
plt.style.use('seaborn-white')
import warnings
warnings.filterwarnings('ignore')
from navec import Navec
from slovnet.model.emb import NavecEmbedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from loguru import logger
import nltk, sys
from stop_words import get_stop_words
from pymystem3 import Mystem
from string import punctuation
from loguru import logger
import tensorflow_hub as hub
import tensorflow_models as tfm
import os
import tensorflow_text as text
sns.set_palette("pastel")

In [34]:
!gdown 1boACz8ab9UytCpi0QhTUFRyS2oyH1MXS

Downloading...
From: https://drive.google.com/uc?id=1boACz8ab9UytCpi0QhTUFRyS2oyH1MXS
To: /Users/glebsokolov/HeadRepo/test_data.csv
100%|██████████████████████████████████████| 53.4k/53.4k [00:00<00:00, 1.04MB/s]


In [13]:
path = '/Users/glebsokolov/HeadRepo/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)
emb = NavecEmbedding(navec)
cosine_similarity = lambda x,y: np.dot(x, y)/(np.linalg.norm(x, 2)*np.linalg.norm(y, 2))

In [5]:
NUM_WORDS = 1000
EMBEDDING_DIM = 300
MAXLEN = 120
PADDING = 'post'
OOV_TOKEN = "<OOV>"
TRAINING_SPLIT = .8

In [6]:
mystem = Mystem()
russian_stopwords = get_stop_words('ru')
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [7]:
path='/Users/glebsokolov/HeadRepo/test_data.csv'
df = pd.read_csv(path)
df.head()
len(df)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


480

In [15]:
tokenizer = Tokenizer(num_words = NUM_WORDS, oov_token=OOV_TOKEN)
texts_combined = [preprocess_text(i) for i in df.iloc[:,3]]
texts_cleared = [i for i in texts_combined if i !='']
tokenizer.fit_on_texts(texts_cleared)
word_index = tokenizer.word_index
sequence = lambda x: tokenizer.texts_to_sequences(x.split(' '))
sequences = pad_sequences([sequence(x) for x in texts_cleared], maxlen=MAXLEN, padding=PADDING)
# keyword = lambda x: [i for i in word_index if word_index[i]==x]

In [52]:
input = torch.tensor([np.array(np.reshape(sequences[i],-1) )for i in range(len(texts_cleared))]).long()
output = tf.convert_to_tensor(emb(input).numpy())

2022-09-03 14:23:44.509962: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [53]:
np.shape(output)

TensorShape([424, 120, 300])

In [54]:
folder_bert = '/Users/glebsokolov/rubert'
tf.io.gfile.listdir(folder_bert)

['bert_model.ckpt', 'bert_config.json', 'vocab.txt', 'bert_model.ckpt.index']

In [55]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [56]:
tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
    vocab_file=os.path.join(folder_bert, "vocab.txt"),
    lower_case=True)
special = tokenizer.get_special_tokens_dict()

In [59]:
i = tf.data.Dataset.from_tensor_slices(texts_cleared)

In [63]:
packer = tfm.nlp.layers.BertPackInputs(
    seq_length=MAXLEN,
    special_tokens_dict = tokenizer.get_special_tokens_dict())

In [68]:
class BertInputProcessor(tf.keras.layers.Layer):
    def __init__(self, tokenizer, packer):
        super().__init__()
        self.tokenizer = tokenizer
        self.packer = packer

    def call(self, inputs):
        to_pack = []
        for sentence in inputs:
            to_pack.append(sentence)

        packed = self.packer(to_pack)

        if 'label' in inputs:
            return packed, inputs['label']
        else:
            return packed

In [70]:
bert_inputs_processor = BertInputProcessor(tokenizer, packer)

In [ ]:
dff = tf.data.experimental.CsvDataset(
    filenames=csv_path,
    record_defaults=[0,"False", "False", 0, 0, 0],
    select_cols=[1, 2, 3,4,5,6],
    field_delim=",",
    header=True)

In [74]:
tf.shape(tf.convert_to_tensor(i))

ValueError: Attempt to convert a value (<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>) to a Tensor.

In [71]:
i.map(bert_inputs_processor).prefetch(1)

ValueError: Exception encountered when calling layer "bert_input_processor_1" (type BertInputProcessor).

in user code:

    File "/var/folders/t0/fw3jvl196_v78v94hzhxhpgw0000gn/T/ipykernel_45158/1029087609.py", line 9, in call  *
        for sentence in inputs:

    ValueError: len requires a non-scalar tensor, got one of shape Tensor("bert_input_processor_1/Shape:0", shape=(0,), dtype=int32)


Call arguments received:
  • inputs=tf.Tensor(shape=(), dtype=string)

In [48]:
texts_cleared

<tf.Tensor: shape=(424,), dtype=string, numpy=
array([b'\xd0\xb0\xd0\xbb\xd0\xb5',
       b'\xd0\xb0\xd0\xbb\xd0\xb5 \xd0\xb7\xd0\xb4\xd1\x80\xd0\xb0\xd0\xb2\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c',
       b'\xd0\xb4\xd0\xbe\xd0\xb1\xd1\x80\xd1\x8b\xd0\xb9',
       b'\xd0\xb7\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c \xd0\xb0\xd0\xbd\xd0\xb3\xd0\xb5\xd0\xbb\xd0\xb8\xd0\xbd\xd0\xb0 \xd0\xba\xd0\xbe\xd0\xbc\xd0\xbf\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x8f \xd0\xb4\xd0\xb8\xd0\xb4\xd0\xb6\xd0\xb8\xd1\x82\xd0\xb0\xd1\x82\xd1\x8c \xd0\xb1\xd0\xb8\xd0\xb7\xd0\xbd\xd0\xb5\xd1\x81 \xd0\xb7\xd0\xb2\xd0\xbe\xd0\xbd\xd0\xb8\xd1\x82\xd1\x8c \xd0\xbf\xd0\xbe\xd0\xb2\xd0\xbe\xd0\xb4 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb4\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xbb\xd0\xb8\xd1\x86\xd0\xb5\xd0\xbd\xd0\xb7\xd0\xb8\xd1\x8f \xd1\x81\xd0\xb5\xd1\x80\xd1\x8b\xd0\xb9 \xd1\x81\xd0\xba\xd0\xbe\xd1\x80\xd0\xbe \xd1\x81\xd1\x80\xd0\xbe\xd0\xba \xd0\xb7\xd0\xb0\xd0\xba\xd0\xb0\xd0\xbd\xd1\x87\xd0\xb8\xd0\xb

In [57]:
texts_cleared = tf.convert_to_tensor(texts_cleared)
t = tf.strings.unicode_decode(texts_cleared, input_encoding='UTF-8')
t = tf.data.Dataset.from_tensor_slices(pad_sequences(t.numpy()))

In [58]:
t = t.map(bert_inputs_processor).prefetch(1)

TypeError: Exception encountered when calling layer "bert_input_processor" (type BertInputProcessor).

in user code:

    File "/var/folders/t0/fw3jvl196_v78v94hzhxhpgw0000gn/T/ipykernel_16752/1573606981.py", line 8, in call  *
        tok1 = self.tokenizer(inputs['sentence1'])

    TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'sentence1'


Call arguments received by layer "bert_input_processor" (type BertInputProcessor):
  • inputs=tf.Tensor(shape=(422,), dtype=int32)

In [ ]:
for reply in df.iloc[:, 3]:
    reply = preprocess_text(reply).split(' ')
    for word in reply:
        try:
            embedding = navec[word]
        except:
            

array([-1.66261181e-01,  4.36078012e-02,  8.01209390e-01, -5.12106478e-01,
        2.20652521e-01, -2.54394650e-01,  1.37822479e-01, -4.04450953e-01,
       -5.55751681e-01, -6.06905110e-02,  2.72250533e-01,  2.69372880e-01,
        3.95740896e-01,  4.15474653e-01,  1.77971542e-01, -2.83912867e-01,
       -2.40878612e-01,  8.44181359e-01,  1.34197801e-01, -6.20480716e-01,
       -1.03538096e-01, -1.62299611e-02, -1.62675217e-01, -3.13942075e-01,
       -4.24463712e-02, -3.59534740e-01, -5.97318172e-01,  4.17822778e-01,
       -2.79673282e-02,  6.92238733e-02, -1.66435927e-01, -1.62600726e-01,
       -3.11053365e-01, -3.78633469e-01,  4.86268848e-02,  1.00024986e+00,
       -3.29663903e-02,  5.77181345e-03,  2.61219759e-02,  2.78724551e-01,
       -6.70146286e-01, -4.35928404e-01,  2.48912144e-02, -1.97291025e-04,
       -6.86537549e-02, -4.08586293e-01,  2.88892329e-01, -1.55857615e-02,
        7.56473169e-02, -2.02433690e-02, -1.38899162e-02,  9.51793268e-02,
        1.49543211e-01,  

In [11]:
emb = NavecEmbedding(navec)

In [31]:
h1 = tf.convert_to_tensor(np.array(navec['здравствуйте'])).numpy()
h2 = tf.convert_to_tensor(navec['привет']).numpy()
print(cosine_similarity)


0.63814735


In [32]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
get_cosine_similarity(h1, h2)

0.6381473